In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.models import load_model
import warnings
import nltk
from nltk.corpus import stopwords
%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('hindi-train.csv')
df.head()

,headline,Label
0,"मेलबर्न एयरपोर्ट पर घंटों फंसे नोवाक जोकोविच, ...",Sports
1,अर्जेंटीना के स्टार फुटबॉलर लियोनल मेसी कोविड-...,Sports
2,"SAI का बड़ा फैसला, कैंपों और प्रैक्टिस सेंटर म...",Sports
3,Pro Kabaddi League: ऑलराउंडर रोहित गूलिया के द...,Sports
4,टेनिस: टाटा ओपन महाराष्ट्र में भारत के युकी भा...,Sports


In [3]:
def count_length():
    df['word_count'] = df['headline'].apply(lambda x: len(str(x).split(" ")))

In [4]:
count_length()

In [5]:
df.head(10)

,headline,Label,word_count
0,"मेलबर्न एयरपोर्ट पर घंटों फंसे नोवाक जोकोविच, ...",Sports,19
1,अर्जेंटीना के स्टार फुटबॉलर लियोनल मेसी कोविड-...,Sports,17
2,"SAI का बड़ा फैसला, कैंपों और प्रैक्टिस सेंटर म...",Sports,18
3,Pro Kabaddi League: ऑलराउंडर रोहित गूलिया के द...,Sports,20
4,टेनिस: टाटा ओपन महाराष्ट्र में भारत के युकी भा...,Sports,13
5,Pro Kabaddi League: पटना पायरेट्स और बंगाल वॉर...,Sports,17
6,"I League: भारतीय फुटबॉल में कोरोना विस्फोट, AI...",Sports,17
7,'दो चैंपियन' के साथ ट्रेनिंग करते नजर आए शिखर ...,Sports,19
8,IND vs SA: जोहानिसबर्ग टेस्ट में पुजारा और रहा...,Sports,21
9,कप्तानी विवाद पर पूर्व पाक कप्तान सलमान बट ने ...,Sports,19


In [6]:
stopwords_hi = ['तुम','मेरी','मुझे','क्योंकि','हम','प्रति','अबकी','आगे','माननीय','शहर','बताएं','कौनसी','क्लिक','किसकी','बड़े','मैं','and','रही','आज','लें','आपके','मिलकर','सब','मेरे','जी','श्री','वैसा','आपका','अंदर', 'अत', 'अपना', 'अपनी', 'अपने', 'अभी', 'आदि', 'आप', 'इत्यादि', 'इन', 'इनका', 'इन्हीं', 'इन्हें', 'इन्हों', 'इस', 'इसका', 'इसकी', 'इसके', 'इसमें', 'इसी', 'इसे', 'उन', 'उनका', 'उनकी', 'उनके', 'उनको', 'उन्हीं', 'उन्हें', 'उन्हों', 'उस', 'उसके', 'उसी', 'उसे', 'एक', 'एवं', 'एस', 'ऐसे', 'और', 'कई', 'कर','करता', 'करते', 'करना', 'करने', 'करें', 'कहते', 'कहा', 'का', 'काफ़ी', 'कि', 'कितना', 'किन्हें', 'किन्हों', 'किया', 'किर', 'किस', 'किसी', 'किसे', 'की', 'कुछ', 'कुल', 'के', 'को', 'कोई', 'कौन', 'कौनसा', 'गया', 'घर', 'जब', 'जहाँ', 'जा', 'जितना', 'जिन', 'जिन्हें', 'जिन्हों', 'जिस', 'जिसे', 'जीधर', 'जैसा', 'जैसे', 'जो', 'तक', 'तब', 'तरह', 'तिन', 'तिन्ह', 'तनह', 'तस', 'तस', 'त', 'थ', 'थ', 'थ', 'दबर', 'दय', 'दसर', 'दसर', 'द', 'दवर', 'न', 'नह', 'न', 'नहयत', 'नच', 'न', 'पर', 'पर', 'पहल', 'पर', 'प', 'फर', 'बन', 'बह', 'बहत', 'बद', 'बल', 'बलकल', 'भ', 'भतर', 'मगर', 'मन', 'म', 'म', 'यद', 'यह', 'यह', 'यह', 'य', 'यह', 'य', 'रख', 'रह', 'रह', 'ऱवस', 'लए', 'लय', 'लकन', 'व', 'वरग', 'वह', 'वह', 'वह', 'वह', 'वल', 'वह', 'व', 'वग़रह', 'सग', 'सकत', 'सकत', 'सबस', 'सभ', 'सथ', 'सबत', 'सभ', 'सर', 'स', 'स', 'ह', 'हआ', 'हई', 'हए', 'ह', 'ह', 'ह', 'हत', 'हत', 'हत', 'हन', 'हन', 'अपन', 'जस', 'हत', 'सभ', 'तह', 'इह', 'दवर', 'इस', 'कह', 'थ', 'उह', 'ओर', 'जह', 'वह', 'अभ', 'बन', 'ह', 'उह', 'उह', 'ह', 'वगरह', 'एस', 'रवस', 'कन', 'नच', 'कफ', 'उस', 'पर', 'भतर', 'ह', 'बह', 'वह', 'कइ', 'यह', 'जह', 'तह', 'कस', 'कइ', 'यह', 'इह', 'जधर', 'इह', 'अद', 'इतयद', 'हइ', 'कनस', 'इसक', 'दसर', 'जह', 'अप', 'कह', 'उनक', 'भ', 'वरग', 'हअ', 'जस', 'नह']
stopwords_en = stopwords.words('english')
punctuations = [".",",","?","/","<",">","!",":",";","'","[","]","{","}","(",")","-","@","#","$","%","^","&","*"]
stopwords_lst = stopwords_hi + stopwords_en + punctuations

In [7]:
df['label_id'] = df.Label.factorize()[0]

def preprocess_text(text):
    text = text.lower()
    text = re.sub('[a-zA-Z0-9]', '',text)
    text = re.sub("((www.[^s]+)|(https?://[^s]+))",'',text)
    text = re.sub("@[^s]+",'',text)
    text = re.sub("[s]+", ' ', text)
    text = re.sub(r"#([^s]+)", r'', text)
    text = text.strip('"')
    return text 

In [8]:
df['headline'] = df.headline.apply(lambda x: preprocess_text(x))
count_length()

In [9]:
df.head(10)

,headline,Label,word_count,label_id
0,"मेलबर्न एयरपोर्ट पर घंटों फंसे नोवाक जोकोविच, ...",Sports,19,0
1,अर्जेंटीना के स्टार फुटबॉलर लियोनल मेसी कोविड-...,Sports,17,0
2,"का बड़ा फैसला, कैंपों और प्रैक्टिस सेंटर में ...",Sports,18,0
3,: ऑलराउंडर रोहित गूलिया के दम पर हरियाणा स्ट...,Sports,20,0
4,टेनिस: टाटा ओपन महाराष्ट्र में भारत के युकी भा...,Sports,13,0
5,: पटना पायरेट्स और बंगाल वॉरियर्स को मिली जी...,Sports,17,0
6,": भारतीय फुटबॉल में कोरोना विस्फोट, ने सप्त...",Sports,17,0
7,'दो चैंपियन' के साथ ट्रेनिंग करते नजर आए शिखर ...,Sports,19,0
8,: जोहानिसबर्ग टेस्ट में पुजारा और रहाणे रहे ...,Sports,21,0
9,कप्तानी विवाद पर पूर्व पाक कप्तान सलमान बट ने ...,Sports,19,0


In [10]:
from indicnlp.tokenize import indic_tokenize
def tokenization(indic_string):
    tokens = []
    for t in indic_tokenize.trivial_tokenize(indic_string):
        tokens.append(t)
    return tokens

In [11]:
df['headline'] = df.headline.apply(lambda x: tokenization(x))
df.head(10)

,headline,Label,word_count,label_id
0,"[मेलबर्न, एयरपोर्ट, पर, घंटों, फंसे, नोवाक, जो...",Sports,19,0
1,"[अर्जेंटीना, के, स्टार, फुटबॉलर, लियोनल, मेसी,...",Sports,17,0
2,"[का, बड़ा, फैसला, ,, कैंपों, और, प्रैक्टिस, से...",Sports,18,0
3,"[:, ऑलराउंडर, रोहित, गूलिया, के, दम, पर, हरिया...",Sports,20,0
4,"[टेनिस, :, टाटा, ओपन, महाराष्ट्र, में, भारत, क...",Sports,13,0
5,"[:, पटना, पायरेट्स, और, बंगाल, वॉरियर्स, को, म...",Sports,17,0
6,"[:, भारतीय, फुटबॉल, में, कोरोना, विस्फोट, ,, न...",Sports,17,0
7,"[', दो, चैंपियन, ', के, साथ, ट्रेनिंग, करते, न...",Sports,19,0
8,"[:, जोहानिसबर्ग, टेस्ट, में, पुजारा, और, रहाणे...",Sports,21,0
9,"[कप्तानी, विवाद, पर, पूर्व, पाक, कप्तान, सलमान...",Sports,19,0


In [12]:
for i in range(len(df)):
    df['headline'][i]=[ele for ele in df['headline'][i] if ele not in (stopwords_lst)]
count_length()
df.head(10)

,headline,Label,word_count,label_id
0,"[मेलबर्न, एयरपोर्ट, घंटों, फंसे, नोवाक, जोकोवि...",Sports,16,0
1,"[अर्जेंटीना, स्टार, फुटबॉलर, लियोनल, मेसी, कोव...",Sports,14,0
2,"[बड़ा, फैसला, कैंपों, प्रैक्टिस, सेंटर, में, प...",Sports,13,0
3,"[ऑलराउंडर, रोहित, गूलिया, दम, हरियाणा, स्टीलर्...",Sports,13,0
4,"[टेनिस, टाटा, ओपन, महाराष्ट्र, में, भारत, युकी...",Sports,10,0
5,"[पटना, पायरेट्स, बंगाल, वॉरियर्स, मिली, जीत, ट...",Sports,9,0
6,"[भारतीय, फुटबॉल, में, कोरोना, विस्फोट, ने, सप्...",Sports,10,0
7,"[दो, चैंपियन, साथ, ट्रेनिंग, नजर, आए, शिखर, धव...",Sports,15,0
8,"[जोहानिसबर्ग, टेस्ट, में, पुजारा, रहाणे, रहे, ...",Sports,15,0
9,"[कप्तानी, विवाद, पूर्व, पाक, कप्तान, सलमान, बट...",Sports,15,0


In [13]:
df['headline'] = df['headline'].str.join(" ")
df.head(10)

,headline,Label,word_count,label_id
0,मेलबर्न एयरपोर्ट घंटों फंसे नोवाक जोकोविच फिर ...,Sports,16,0
1,अर्जेंटीना स्टार फुटबॉलर लियोनल मेसी कोविड टेस...,Sports,14,0
2,बड़ा फैसला कैंपों प्रैक्टिस सेंटर में पहुंचने ...,Sports,13,0
3,ऑलराउंडर रोहित गूलिया दम हरियाणा स्टीलर्स ने य...,Sports,13,0
4,टेनिस टाटा ओपन महाराष्ट्र में भारत युकी भांबरी...,Sports,10,0
5,पटना पायरेट्स बंगाल वॉरियर्स मिली जीत टाइटंस त...,Sports,9,0
6,भारतीय फुटबॉल में कोरोना विस्फोट ने सप्ताह लिए...,Sports,10,0
7,दो चैंपियन साथ ट्रेनिंग नजर आए शिखर धवन रोहित ...,Sports,15,0
8,जोहानिसबर्ग टेस्ट में पुजारा रहाणे रहे फ्लॉप ट...,Sports,15,0
9,कप्तानी विवाद पूर्व पाक कप्तान सलमान बट ने राह...,Sports,15,0


In [14]:
X = df['headline'].values
Y = df['label_id']
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [15]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20, random_state=42)
max_words = 10000
max_len = 200
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(7,name='out_layer')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model
model = RNN()

In [16]:
test_classes = []
for i in range(len(Y_test)):
    for j in range(len(Y_test[i])):
        test_classes.append(Y_test[i][j]) 

In [17]:
model.compile(loss='sparse_categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
history = model.fit(sequences_matrix,Y_train,batch_size=256,epochs=10,
          validation_split=0.2)

Epoch 1/10
94/94 [==============================] - 55s 551ms/step - loss: 1.2042 - accuracy: 0.5022 - val_loss: 0.5319 - val_accuracy: 0.8153
Epoch 2/10
94/94 [==============================] - 59s 624ms/step - loss: 0.4222 - accuracy: 0.8382 - val_loss: 0.3398 - val_accuracy: 0.8603
Epoch 3/10
94/94 [==============================] - 65s 693ms/step - loss: 0.3241 - accuracy: 0.8723 - val_loss: 0.2757 - val_accuracy: 0.8772
Epoch 4/10
94/94 [==============================] - 52s 557ms/step - loss: 0.2346 - accuracy: 0.8906 - val_loss: 0.2586 - val_accuracy: 0.8863
Epoch 5/10
94/94 [==============================] - 51s 544ms/step - loss: 0.2091 - accuracy: 0.8988 - val_loss: 0.2587 - val_accuracy: 0.8858
Epoch 6/10
94/94 [==============================] - 56s 601ms/step - loss: 0.1931 - accuracy: 0.9028 - val_loss: 0.2548 - val_accuracy: 0.8875
Epoch 7/10
94/94 [==============================] - 54s 572ms/step - loss: 0.2394 - accuracy: 0.9001 - val_loss: 0.2498 - val_accuracy: 0.8863

In [18]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
accr = model.evaluate(test_sequences_matrix,Y_test)
print('Accuracy: {:0.2f}'.format(accr[1]))

235/235 [==============================] - 8s 36ms/step - loss: 0.2994 - accuracy: 0.8727
Accuracy: 0.87


In [19]:
model.save('HI_LSTM.h5')

In [20]:
model = load_model('HI_LSTM.h5')

In [20]:
predicted_categories = []
preds = model.predict(test_sequences_matrix)
for i in preds:
    pred_classes = np.argsort(i)[-1:][::-1]
    predicted_categories.append(pred_classes[0])

In [21]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
report = classification_report(test_classes, predicted_categories)
print(report)

              precision    recall  f1-score   support

           0       0.98      0.78      0.87      1523
           1       0.73      0.83      0.78      1493
           2       0.95      0.96      0.95      1477
           3       0.95      0.98      0.96      1468
           4       0.80      0.82      0.81      1539

    accuracy                           0.87      7500
   macro avg       0.88      0.87      0.87      7500
weighted avg       0.88      0.87      0.87      7500



In [22]:
mat = confusion_matrix(test_classes, predicted_categories)
mat

array([[1191,  182,   16,   44,   90],
       [   2, 1234,   37,    3,  217],
       [   3,   26, 1421,   22,    5],
       [   6,    1,   21, 1432,    8],
       [  17,  236,    8,   11, 1267]], dtype=int64)

In [23]:
id_to_category = {0:'sports', 1:'tech', 2:'business', 3:'politics',
                  4:'entertainment'}

In [24]:
sample_text1 = ["सेंसेक्स 1129 पॉइंट्स लुढ़का, रिलायंस का शेयर 4.5% टूटा, मार्केट कैप 2 घंटे में 7.88 लाख करोड़ घटा"]

txts = tok.texts_to_sequences(sample_text1)
txts = sequence.pad_sequences(txts, maxlen=max_len)
preds = model.predict(txts)[0]
pred_classes = np.argsort(preds)[-1:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

for c, p in zip(classes, props):
    print("{} {:.2f} %".format(c,p*100))

business 100.00 %


In [25]:
sample_text2 = ["शादी की खबरों के बीच कटरीना कैफ पर भड़के विक्की कौशल, दोनों के बीच हुआ झगड़ा! जानें वजह"]

txts = tok.texts_to_sequences(sample_text2)
txts = sequence.pad_sequences(txts, maxlen=max_len)
preds = model.predict(txts)[0]
pred_classes = np.argsort(preds)[-1:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

for c, p in zip(classes, props):
    print("{} {:.2f} %".format(c,p*100))

entertainment 100.00 %


In [29]:
sample_text3 = ["पीएम मोदी का आज से तीन दिवसीय गुजरात दौरा, मारीशस के प्रधानमंत्री और डब्ल्यूएचओ प्रमुख भी रहेंगे मौजूद"]

txts = tok.texts_to_sequences(sample_text3)
txts = sequence.pad_sequences(txts, maxlen=max_len)
preds = model.predict(txts)[0]
pred_classes = np.argsort(preds)[-1:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

for c, p in zip(classes, props):
    print("{} {:.2f} %".format(c,p*100))

politics 100.00 %


In [30]:
sample_text4 = ["भारत पहुंची पाकिस्तान की टीम, 24 नवंबर से भुवनेश्वर में खेला जाएगा वर्ल्ड कपाा"]

txts = tok.texts_to_sequences(sample_text4)
txts = sequence.pad_sequences(txts, maxlen=max_len)
preds = model.predict(txts)[0]
pred_classes = np.argsort(preds)[-1:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

for c, p in zip(classes, props):
    print("{} {:.2f} %".format(c,p*100))

sports 99.98 %


In [31]:
sample_text5 = ["सावधान! अगर ये 7 ऐप्स आपके फोन में हों तो तुरंत करें डिलीट, बन सकते हैं खतरा"]

txts = tok.texts_to_sequences(sample_text5)
txts = sequence.pad_sequences(txts, maxlen=max_len)
preds = model.predict(txts)[0]
pred_classes = np.argsort(preds)[-1:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

for c, p in zip(classes, props):
    print("{} {:.2f} %".format(c,p*100))

tech 99.98 %
